In [1]:
import enoki as ek
import mitsuba
mitsuba.set_variant('gpu_autodiff_rgb')

from mitsuba.core import Thread, Float
from mitsuba.core.xml import load_file
from mitsuba.python.util import traverse
from mitsuba.python.autodiff import render, write_bitmap

# Load the Cornell Box
Thread.thread().file_resolver().append('cbox/')

# scene = load_file('cbox/cbox_path.xml')
# scene = load_file('cbox/cbox_direct.xml')
# scene = load_file('cbox/cbox_path_tc.xml')
# scene = load_file('cbox/cbox_direct_tc.xml')
scene = load_file('cbox/cbox_path_nee.xml')

# Find differentiable scene parameters
params = traverse(scene)

wall_res = params['wall_bsdf.reflectance.resolution']
wall_data = Float(params['wall_bsdf.reflectance.data'])
print(wall_res, wall_data)

params.keep(['wall_bsdf.reflectance.data'])

2021-11-29 21:41:38 INFO main [optix_api.cpp:56] Dynamic loading of the Optix library ..

2021-11-29 21:41:38 INFO main [xml.cpp:1221] Loading XML file "cbox\cbox_path_nee.xml" ..

2021-11-29 21:41:38 INFO main [xml.cpp:1222] Using variant "gpu_autodiff_rgb"

2021-11-29 21:41:38 INFO main [xml.cpp:355] "cbox\cbox_path_nee.xml": in-memory version upgrade (v2.0.0 -> v2.2.1) ..

2021-11-29 21:41:38 INFO main [PluginManager] Loading plugin "plugins\srgb.dll" ..

...

2021-11-29 21:41:38 INFO main [PluginManager] Loading plugin "plugins\perspective.dll" ..

2021-11-29 21:41:38 INFO main [PluginManager] Loading plugin "plugins\area.dll" ..

2021-11-29 21:41:38 INFO main [PluginManager] Loading plugin "plugins\uniform.dll" ..

2021-11-29 21:41:38 INFO main [Scene] Building scene in OptiX ..

[16, 16] [0, 0, 0, 0, 0, .. 758 skipped .., 0.520996, 0.799103, 0.577581, 0.520996, 0.799103]


In [2]:
image_ref = render(scene, spp=256)
crop_size = scene.sensors()[0].film().crop_size()
write_bitmap('out/out_ref.png', image_ref, crop_size)

![out_ref.png](out/out_ref.png)

In [3]:
params['wall_bsdf.reflectance.data'] = ek.full(Float, 1.0, len(wall_data))
params.update()

from mitsuba.python.autodiff import Adam
opt = Adam(params, lr=0.1)

image = render(scene, spp=8)
crop_size = scene.sensors()[0].film().crop_size()
write_bitmap('out/out_start.png', image, crop_size)
write_bitmap('out/wall_start.png', params['wall_bsdf.reflectance.data'],
                 (wall_res[0], wall_res[1]))

for it in range(200):
    # Perform a differentiable rendering of the scene
    image = render(scene, optimizer=opt, unbiased=True, spp=1)

    # Objective: MSE between 'image' and 'image_ref'
    ob_val = ek.hsum(ek.sqr(image - image_ref)) / len(image)

    # Back-propagate errors to input parameters
    ek.backward(ob_val)

    # Optimizer: take a gradient step
    opt.step()

    err_ref = ek.hsum(ek.sqr(wall_data - params['wall_bsdf.reflectance.data']))

    print('%g,%g' % (ob_val[0], err_ref[0]))

image = render(scene, spp=8)
crop_size = scene.sensors()[0].film().crop_size()
write_bitmap('out/out_end.png', image, crop_size)
write_bitmap('out/wall_end.png', params['wall_bsdf.reflectance.data'],
                 (wall_res[0], wall_res[1]))

0.190929,319.126
0.16333,299.092
0.181248,266.123
0.154855,236.294
0.191595,208.768
0.188959,184.368
0.195578,163.781
0.225414,146.487
0.210091,134.184
0.130853,127.142
0.125745,124.93
0.136859,126.5
0.113117,131.652
0.145669,139.461
0.11776,150.157
0.110125,163.18
0.0959142,178.025
0.120677,195.015
0.134207,214.326
0.12168,235.087
0.118898,256.95
0.125804,279.843
0.106032,302.187
0.10438,323.575
0.106236,346.945
0.105565,370.294
0.12533,393.712
0.108627,416.055
0.130136,435.545
0.128209,454.732
0.109019,471.69
0.110106,487.445
0.141131,503.148
0.139307,519.659
0.122747,534.479
0.153601,547.795
0.125226,559.953
0.151914,572.029
0.131171,581.596
0.122191,590.388
0.133401,598.299
0.118317,607.227
0.123638,614.479
0.140247,620.199
0.117539,624.991
0.130259,627.658
0.122779,630.816
0.145152,634.066
0.131587,637.798
0.121316,642.977
0.146217,647.222
0.121168,651.709
0.156225,656.979
0.120625,662.659
0.148415,667.31
0.138391,670.324
0.132824,672.375
0.113634,671.28
0.135116,670.857
0.117065,

![start](out/out_start.png)
![end](out/out_end.png)
![start](out/wall_start.png)
![end](out/wall_end.png)